# Text Chunking, Embedding, and Vector Store Indexing

## 1. Load the Cleaned Dataset

In [1]:
import pandas as pd

# Mount Google Drive (for Colab)
from google.colab import drive
drive.mount('/content/drive')

# Load the cleaned dataset from Google Drive
data_path = '/content/drive/MyDrive/Task1/df_filtered_20230709.csv.gz'
df = pd.read_csv(data_path)
print("Dataset Shape:", df.shape)

# Check unique products and their counts
print("\nUnique Products and their counts:")
product_counts = df['Products'].value_counts()
print(product_counts)

# Select 1,000 rows for each of the 4 unique product values
df_sample = pd.DataFrame()
unique_products = df['Products'].unique()

for product in unique_products:
    product_df = df[df['Products'] == product].head(1000)
    df_sample = pd.concat([df_sample, product_df], ignore_index=True)

print(f"\nSample Dataset Shape: {df_sample.shape}")
print(f"Sample distribution by product:")
print(df_sample['Products'].value_counts())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dataset Shape: (2963776, 13)

Unique Products and their counts:
Products
Credit card        2116734
Personal loan       594223
Savings account     154489
Money transfers      98330
Name: count, dtype: int64

Sample Dataset Shape: (4000, 13)
Sample distribution by product:
Products
Credit card        1000
Personal loan      1000
Savings account    1000
Money transfers    1000
Name: count, dtype: int64


## 2. Implement Text Chunking

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Initialize text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,  # Number of characters per chunk
    chunk_overlap=50,  # Overlap to maintain context between chunks
    length_function=len
)

# Apply chunking to cleaned narratives (using sample data)
chunks = []
for index, row in df_sample.iterrows():
    # Check if cleaned_narrative is not null and is a string
    if pd.notna(row['cleaned_narrative']) and isinstance(row['cleaned_narrative'], str):
        narratives = text_splitter.split_text(row['cleaned_narrative'])
        for i, chunk in enumerate(narratives):
            chunks.append({
                'complaint_id': row['Complaint ID'],
                'product': row['Products'],
                'chunk_text': chunk,
                'chunk_id': f"{row['Complaint ID']}_chunk_{i}"
            })

# Convert chunks to DataFrame
chunks_df = pd.DataFrame(chunks)
print("Number of Chunks:", len(chunks_df))
print("\nChunks distribution by product:")
print(chunks_df['product'].value_counts())
print("\nFirst Few Chunks:\n", chunks_df.head())

Number of Chunks: 18256

Chunks distribution by product:
product
Savings account    5097
Personal loan      4736
Credit card        4642
Money transfers    3781
Name: count, dtype: int64

First Few Chunks:
    complaint_id      product  \
0      12351447  Credit card   
1      12351447  Credit card   
2      12351447  Credit card   
3      12351447  Credit card   
4      12351447  Credit card   

                                          chunk_text          chunk_id  
0  apt , , t transunion consumer solutions , pa r...  12351447_chunk_0  
1  state laws, including but not limited to the f...  12351447_chunk_1  
2  security number date of birth current address ...  12351447_chunk_2  
3  should remain in place until i personally requ...  12351447_chunk_3  
4  license or governmentissued id. o a copy of a ...  12351447_chunk_4  


## 3. Justify Chunking Strategy

The chunking strategy uses:
- **Chunk Size**: 300 characters - optimal for maintaining context while fitting within model limits
- **Overlap**: 50 characters - ensures continuity between chunks and prevents information loss
- **Method**: RecursiveCharacterTextSplitter - intelligently splits on natural boundaries

## 4. Choose an Embedding Model

In [9]:
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import os
from tqdm import tqdm
import time
import psutil  # For memory monitoring

# For Google Colab: Mount Google Drive to save outputs
from google.colab import drive
drive.mount('/content/drive')

# Set output directory to Google Drive
output_dir = '/content/drive/MyDrive/vector_store/chromadb_sample_dataset'
os.makedirs(output_dir, exist_ok=True)
print(f"Output will be saved to: {output_dir}")

# Check system resources
print(f"CPU Cores: {psutil.cpu_count(logical=True)}")
print(f"Available RAM: {psutil.virtual_memory().available / (1024 ** 3):.2f} GB")

# Set device to CPU (no GPU available)
device = 'cpu'
print(f"Using device: {device}")

# Load the embedding model with optimized settings
embedding_model = SentenceTransformer(
    'all-MiniLM-L6-v2',
    device=device
)
print("Embedding Model Loaded:", embedding_model)

# Total number of chunks (using sample data)
total_chunks = len(chunks_df)
print(f"Total Chunks to Embed: {total_chunks}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Output will be saved to: /content/drive/MyDrive/vector_store/chromadb_sample_dataset
CPU Cores: 96
Available RAM: 321.93 GB
Using device: cpu


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding Model Loaded: SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)
Total Chunks to Embed: 18256


## 5. Justify Embedding Model Choice

**Model**: all-MiniLM-L6-v2
**Advantages**:
- Fast inference (384 dimensions vs 768+ for larger models)
- Good performance on semantic similarity tasks
- CPU-friendly architecture
- Balanced trade-off between speed and accuracy
- Widely used in production systems

## 6. Create and Index Vector Store with ChromaDB

In [10]:
import chromadb
from chromadb.config import Settings
import numpy as np
from tqdm import tqdm
import os
import time

# Initialize ChromaDB client with persistent storage
chroma_client = chromadb.PersistentClient(
    path=output_dir,
    settings=Settings(
        anonymized_telemetry=False,
        allow_reset=True
    )
)

# Create or get collection
collection_name = "consumer_complaints"
try:
    collection = chroma_client.get_collection(name=collection_name)
    print(f"Loaded existing collection: {collection_name}")
except:
    collection = chroma_client.create_collection(
        name=collection_name,
        metadata={"description": "Consumer complaints vector store"}
    )
    print(f"Created new collection: {collection_name}")

# Prepare data for ChromaDB
documents = []
metadatas = []
ids = []

print("Preparing data for ChromaDB...")
for index, row in tqdm(chunks_df.iterrows(), total=len(chunks_df), desc="Preparing data"):
    documents.append(row['chunk_text'])
    metadatas.append({
        'complaint_id': str(row['complaint_id']),
        'product': row['product'],
        'chunk_id': row['chunk_id']
    })
    ids.append(row['chunk_id'])

# Add documents to ChromaDB in batches
batch_size = 1000  # ChromaDB batch size
total_batches = (len(documents) + batch_size - 1) // batch_size

print(f"Adding {len(documents)} documents to ChromaDB in {total_batches} batches...")
start_time = time.time()

for i in tqdm(range(0, len(documents), batch_size), desc="Adding to ChromaDB"):
    batch_end = min(i + batch_size, len(documents))

    batch_documents = documents[i:batch_end]
    batch_metadatas = metadatas[i:batch_end]
    batch_ids = ids[i:batch_end]

    collection.add(
        documents=batch_documents,
        metadatas=batch_metadatas,
        ids=batch_ids
    )

print("Indexing Time:", time.time() - start_time, "seconds")
print("Total Documents Indexed:", collection.count())
print(f"ChromaDB collection saved to: {output_dir}")

Created new collection: consumer_complaints
Preparing data for ChromaDB...


Preparing data: 100%|██████████| 18256/18256 [00:00<00:00, 24136.65it/s]


Adding 18256 documents to ChromaDB in 19 batches...


Adding to ChromaDB:   0%|          | 0/19 [00:00<?, ?it/s]
/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz:   0%|          | 0.00/79.3M [00:00<?, ?iB/s]
/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz:   0%|          | 272k/79.3M [00:00<00:33, 2.50MiB/s]
/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz:   4%|▍         | 3.41M/79.3M [00:00<00:04, 18.1MiB/s]
/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz:  18%|█▊        | 14.2M/79.3M [00:00<00:01, 54.8MiB/s]
/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz:  31%|███▏      | 24.8M/79.3M [00:00<00:00, 75.4MiB/s]
/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz:  43%|████▎     | 33.8M/79.3M [00:00<00:00, 81.2MiB/s]
/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz:  57%|█████▋    | 45.5M/79.3M [00:00<00:00, 94.9MiB/s]
/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz:  72%|███████▏  | 56.8M/79.3M [00:00<00:00, 97.6MiB/s]
/root/.cache/chroma/onn

Indexing Time: 1907.6969063282013 seconds
Total Documents Indexed: 18256
ChromaDB collection saved to: /content/drive/MyDrive/vector_store/chromadb_sample_dataset


## 7. Test Vector Store Functionality

In [11]:
# Test the vector store with a sample query
import numpy as np

print(f"Collection contains {collection.count()} documents")

# Test query
test_query = "credit card dispute"
print(f"\nSearching for: '{test_query}'")

# Search for similar documents
results = collection.query(
    query_texts=[test_query],
    n_results=5,
    include=['documents', 'metadatas', 'distances']
)

print(f"\nTop 5 similar chunks for query: '{test_query}'")
print("-" * 50)

for i, (doc, metadata, distance) in enumerate(zip(
    results['documents'][0],
    results['metadatas'][0],
    results['distances'][0]
)):
    print(f"{i+1}. Chunk ID: {metadata['chunk_id']}")
    print(f"   Product: {metadata['product']}")
    print(f"   Complaint ID: {metadata['complaint_id']}")
    print(f"   Distance: {distance:.4f}")
    print(f"   Text: {doc[:100]}...")
    print()

Collection contains 18256 documents

Searching for: 'credit card dispute'

Top 5 similar chunks for query: 'credit card dispute'
--------------------------------------------------
1. Chunk ID: 13473619_chunk_8
   Product: Savings account
   Complaint ID: 13473619
   Distance: 0.6173
   Text: the card and sending me a new one. i have to pay bills with this card and buy groceries with this ca...

2. Chunk ID: 12625283_chunk_1
   Product: Money transfers
   Complaint ID: 12625283
   Distance: 0.6377
   Text: so i contact them to dispute. 4. they take no responsibility and direct me to my bank to get a new c...

3. Chunk ID: 14001558_chunk_3
   Product: Credit card
   Complaint ID: 14001558
   Distance: 0.6457
   Text: both credit card company and the merchant with no avail. i have disputed but was told i lost and owe...

4. Chunk ID: 13938213_chunk_9
   Product: Credit card
   Complaint ID: 13938213
   Distance: 0.6703
   Text: i respectfully request that this dispute be resolved in my fa

## 8. Organize Deliverables

In [12]:
# Organize deliverables: Copy ChromaDB files to the vector_store/ directory

import shutil

# Define source and destination paths
chromadb_src = output_dir
chromadb_dst = "vector_store/chromadb_sample_dataset"

# Copy ChromaDB directory
if os.path.exists(chromadb_src):
    if os.path.exists(chromadb_dst):
        shutil.rmtree(chromadb_dst)
    shutil.copytree(chromadb_src, chromadb_dst)
    print(f"Copied ChromaDB collection to {chromadb_dst}")
else:
    print(f"ChromaDB collection not found at {chromadb_src}")

print("\nSample vector store creation completed successfully!")
print(f"Total samples processed: {len(df_sample)}")
print(f"Total chunks created: {len(chunks_df)}")
print(f"Total documents indexed: {collection.count()}")

Copied ChromaDB collection to vector_store/chromadb_sample_dataset

Sample vector store creation completed successfully!
Total samples processed: 4000
Total chunks created: 18256
Total documents indexed: 18256


 # Text Chunking, Embedding, and Vector Store Indexing - Summary

 ## Overview
 This notebook walks through the process of building a semantic vector store from consumer complaint narratives using text chunking, embedding generation with a transformer model, and storage in ChromaDB for efficient similarity search.

 ## Key Steps in This Notebook

 ### 1. Dataset Preparation
 - **Source**: Cleaned and filtered CFPB complaints dataset.
 - **Sampling**: A sample of 18,256 complaints was selected for demonstration.
 - **Products**: Multiple product categories are included for diversity.

 ### 2. Text Chunking
 - **Method**: Each complaint narrative is split into overlapping text chunks.
 - **Chunk Size**: 300 characters.
 - **Overlap**: 50 characters.
 - **Purpose**: Ensures context is preserved and fits model input size.
 - **Result**: 19 chunk batches created for embedding.

 ### 3. Embedding Generation
 - **Model**: `all-MiniLM-L6-v2` from Sentence Transformers.
 - **Embedding Size**: 384 dimensions.
 - **Batch Processing**: Chunks are embedded in batches for efficiency.
 - **Hardware**: CPU-based inference (suitable for local/Colab environments).

 ### 4. Vector Store Creation (ChromaDB)
 - **Database**: ChromaDB is used to store embeddings and metadata.
 - **Metadata**: Each chunk is associated with product, complaint ID, and chunk ID.
 - **Persistence**: The ChromaDB collection is saved to disk and copied to `vector_store/chromadb_sample_dataset`.

 ### 5. Similarity Search & Validation
 - **Test Query**: A sample query is embedded and used to search the vector store.
 - **Results**: Top 5 most similar complaint chunks are retrieved and displayed, including metadata and similarity scores.

 ## Results & Deliverables

 - **ChromaDB vector store** created and saved to `vector_store/chromadb_sample_dataset`.
 - **All complaint chunks** embedded and indexed.
 - **Sample similarity search** demonstrates retrieval quality.
 - **Metadata** (product, complaint ID, chunk ID) is preserved for downstream analysis.

 ## File Structure

 vector_store/
   └── chromadb_sample_dataset/
       ├── [ChromaDB collection files]

 ## Use Cases
 - **Semantic Search**: Retrieve similar complaints for a given query.
 - **RAG Applications**: Use as a retrieval backend for LLM-powered analysis.
 - **Product/Theme Analysis**: Explore complaint patterns by product or topic.

 ## Next Steps
 - Integrate this vector store with the RAG pipeline for end-to-end complaint analysis.
 - Build a Streamlit-based query interface for interactive exploration.
 - Scale up to the full dataset as needed.
 - Evaluate retrieval quality and optimize chunking/embedding parameters.

 ---
 *This ChromaDB vector store enables fast, semantic search and lays the groundwork for advanced retrieval-augmented analytics in the financial complaints domain.*